# NHS 111 Programme
https://www.england.nhs.uk/statistics/statistical-work-areas/nhs-111-minimum-data-set/nhs-111-minimum-data-set-2017-18/

In [25]:
%%html
<img src='http://www.staffordsurroundsccg.nhs.uk/images/AID_1521_NHS-111-cannock-2_02.09.15_V1_resized.jpg' height="75px" width="50%">

### Objective 
Visualise time series for 111 metrics

# Set Up

In [26]:
import pandas as pd
import numpy as np

In [27]:
url = 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2017/12/NHS-111-MDS-time-series-to-2017-November-v2.xlsx'
df = pd.read_excel(url, sheet_name='Raw', skiprows=5, header=0, )#, sheet_name='All Attendances - Male', skiprows=3, header=0)
df.rename(columns={'Unnamed: 0':'Concat', 'Unnamed: 1':'Region', 'Unnamed: 2':'Provider Code', 'Unnamed: 3':'Date', 'Unnamed: 4':'Code', 'Unnamed: 5':'Area'}, inplace=True)

In [28]:
df.head()

,Concat,Region,Provider Code,Date,Code,Area,Population,Total calls offered,No of abandoned calls,No calls answered,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18
0,111AA440513,ME,DHU,2010-12-01,111AA4,NOTTINGHAM CITY NHS 111,299753.0,5782.0,311.0,5253.0,...,NaN,April,Apr,1343.1,3189.53,18884.4,36029.8,37696.5,39741.2,45968
1,111AA440544,ME,DHU,2011-01-01,111AA4,NOTTINGHAM CITY NHS 111,303899.0,5865.0,164.0,5473.0,...,NaN,May,May,1162.26,3226.19,18739.9,35891.4,38199.3,42135.5,42161.2
2,111AA440575,ME,DHU,2011-02-01,111AA4,NOTTINGHAM CITY NHS 111,303899.0,3567.0,8.0,3493.0,...,NaN,June,Jun,1121.07,3493.87,18423.9,32792.8,32903.6,37499.7,38835.8
3,111AA440603,ME,DHU,2011-03-01,111AA4,NOTTINGHAM CITY NHS 111,303899.0,4942.0,66.0,4714.0,...,NaN,July,Jul,1129.03,3096.84,18755.7,31784.7,32782.2,39966.8,40699.9
4,111AA440634,ME,DHU,2011-04-01,111AA4,NOTTINGHAM CITY NHS 111,303899.0,5797.0,110.0,5532.0,...,NaN,August,Aug,1143.97,2795.32,17687.7,31731.4,34241.9,36071.2,37615.1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604 entries, 0 to 2603
Data columns (total 72 columns):
Concat                                                                         2603 non-null object
Region                                                                         2604 non-null object
Provider Code                                                                  2603 non-null object
Date                                                                           2603 non-null datetime64[ns]
Code                                                                           2603 non-null object
Area                                                                           2603 non-null object
Population                                                                     2603 non-null float64
Total calls offered                                                            2603 non-null float64
No of abandoned calls                                                          2603

# Visualise the data

In [30]:
%%html
<img src='https://user-images.githubusercontent.com/1280389/30125654-18858b44-9307-11e7-96bc-c37c15b2be51.png' height="75px" width="25%">
<a href="url">'https://plot.ly'</a>

### Viz with df without index

In [31]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from IPython.display import clear_output # used to stop the duplicate charts

import ipywidgets as widgets
from ipywidgets.widgets import interact, interactive

In [32]:
metric_options = ['Population', 'Total calls offered', 'No calls answered', 'Calls answered within 60 secs','Ambulance dispatches']
dimension_options = ['Area','Region','Provider Code']

In [34]:
plotly.offline.init_notebook_mode(connected=False)


def update_chart(dimension_element_picker, dimension_picker):

    # if all is selected in the dimension_element_picker then do not include a dimension in the group by clause
    if dimension_element_picker == 'All':
        df_grouped = df.groupby(by=['Date'], as_index=False).sum()
    else:
        df_filtered = df[df[dimension_picker] == dimension_element_picker]
        df_grouped = df_filtered.groupby(by=[dimension_picker,'Date'], as_index=False).sum()

    # Could allow users to select metrics
    metric_options_selected = metric_options
    
    data = [go.Scatter(
        x = df_grouped['Date'],
        y = df_grouped[metric_options_selected[i]],
        mode = 'lines',
        name = metric_options_selected[i],
#         text = df_grouped[metric_options_selected[i]],
        opacity = 0.8,


    ) for i in range(0, len(metric_options_selected))]

    layout = go.Layout(
        barmode='group',#'group', # switch between stack and group
        title='<b>NHS 111 calls where  </b>'+ dimension_picker+' = '+dimension_element_picker,
        yaxis = dict(
            type = 'log', # switches to a logarythmic scale
            title='<i>Volume</i>'
        ),
#         xaxis=dict(
#             title='<i>Date</i>'
#         )      
    )

    fig = go.Figure(data=data, layout=layout)

    plotly.offline.iplot(fig) # shows the plot offline


Initial_widget_dimension_elements_filter = list(df['Area'].unique())
Initial_widget_dimension_elements_filter.insert(0,'All')

widget_dimension_options = widgets.ToggleButtons(options=dimension_options, description='Options:')
widget_dimension_elements_filter = widgets.Dropdown(options=Initial_widget_dimension_elements_filter, description='Filter:')

def update_dimension_elements_filter(change):
    
    # work around to clear cell output 
    clear_output()
#     display(accordion) # only needed due to bug in ipywidgets version 7 (reverted to version 6 to display widgets on nbvwier)
    
    option_list = list(df[change['new']].unique())
    option_list.insert(0,'All')
    widget_dimension_elements_filter.options = option_list
    update_chart(dimension_element_picker='All', dimension_picker=widget_dimension_options.value)
    
widget_dimension_options.observe(update_dimension_elements_filter, 'value')

def widget_dimension_elements_filter_update(change):
    
    # work around to clear cell output 
    clear_output()
#     display(accordion) # only needed due to bug in ipywidgets version 7 (reverted to version 6 to display widgets on nbvwier)
    
    widget_dimension_elements_filter_selected = change['new']
    update_chart(dimension_element_picker=widget_dimension_elements_filter_selected, dimension_picker=widget_dimension_options.value)

widget_dimension_elements_filter.observe(widget_dimension_elements_filter_update, 'value')

# display(widget_dimension_options, widget_dimension_elements_filter)

# SWITCH from simply displaying widgets to showing them via an accordian
accordion = widgets.Accordion(children=[widget_dimension_options, widget_dimension_elements_filter])
accordion.set_title(0, 'Choose a Dimension To filter by')
accordion.set_title(1, 'Choose a member of that dimension to filter by')
display(accordion)

update_chart(dimension_element_picker='All', dimension_picker='Area')

# Notes for graph

* Uses a logarythmic y axis to help show percent change of metrics
* Use the acodian user interface to change how to slice NHS 111 Metrics

### Back Ground

NHS 111 is available 24 hours a day, 7 days a week, 365 days a year to respond to people’s health care needs when:
*	it’s not a life threatening situation, and therefore is less urgent than a 999 call
*	the GP isn’t an option, for instance when the caller is away from home
*	the caller feels they cannot wait and is simply unsure of which service they require
*	the caller requires reassurance about what to do next.

NHS 111 answers the call, assesses the caller’s needs and determines the most appropriate course of action, including:
*	for callers facing an emergency, an ambulance will be despatched without delay
*	where a face to face consultation is required, an appointment will be booked or the caller will be referred to the service that has the appropriate skills and resources to meet their needs in the required timeframe
*	for callers who do not require a face-to-face consultation, information, advice and reassurance will be provided
*	where the call is outside the scope of NHS 111, the caller will be signposted to an alternative service

### Source
https://www.england.nhs.uk/statistics/statistical-work-areas/nhs-111-minimum-data-set/

### Bugs
* Visualisation duplicates charts rather than updates them


# TO DO 

add widgets to:
* solve duplicate / multiple plots issue
* select multiple metrics,


* also add some annotation and better hover over



http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

# DEV